In [1]:
# Load libraries
from sklearn.datasets import load_iris
import numpy as np
import pandas as pd
import sys
import random 

In [2]:
# Load the test data
iris = load_iris()
# Create a pandas dataframe
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['species'] = iris.target
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [3]:
# extract a matriz from the df
iris_matrix = iris_df.values
iris_matrix

array([[5.1, 3.5, 1.4, 0.2, 0. ],
       [4.9, 3. , 1.4, 0.2, 0. ],
       [4.7, 3.2, 1.3, 0.2, 0. ],
       [4.6, 3.1, 1.5, 0.2, 0. ],
       [5. , 3.6, 1.4, 0.2, 0. ],
       [5.4, 3.9, 1.7, 0.4, 0. ],
       [4.6, 3.4, 1.4, 0.3, 0. ],
       [5. , 3.4, 1.5, 0.2, 0. ],
       [4.4, 2.9, 1.4, 0.2, 0. ],
       [4.9, 3.1, 1.5, 0.1, 0. ],
       [5.4, 3.7, 1.5, 0.2, 0. ],
       [4.8, 3.4, 1.6, 0.2, 0. ],
       [4.8, 3. , 1.4, 0.1, 0. ],
       [4.3, 3. , 1.1, 0.1, 0. ],
       [5.8, 4. , 1.2, 0.2, 0. ],
       [5.7, 4.4, 1.5, 0.4, 0. ],
       [5.4, 3.9, 1.3, 0.4, 0. ],
       [5.1, 3.5, 1.4, 0.3, 0. ],
       [5.7, 3.8, 1.7, 0.3, 0. ],
       [5.1, 3.8, 1.5, 0.3, 0. ],
       [5.4, 3.4, 1.7, 0.2, 0. ],
       [5.1, 3.7, 1.5, 0.4, 0. ],
       [4.6, 3.6, 1. , 0.2, 0. ],
       [5.1, 3.3, 1.7, 0.5, 0. ],
       [4.8, 3.4, 1.9, 0.2, 0. ],
       [5. , 3. , 1.6, 0.2, 0. ],
       [5. , 3.4, 1.6, 0.4, 0. ],
       [5.2, 3.5, 1.5, 0.2, 0. ],
       [5.2, 3.4, 1.4, 0.2, 0. ],
       [4.7, 3

# Common functions

## Function to calculate Euclidean Distance

In [4]:
def calculate_euclidean_distance(np_array1, np_array2):
    """
    Args:
        np_array1 (np.ndarray): 1-dimensional array with n elements 
        np_array2 (np.ndarray): 1-dimensional array with n elements 
    """
    # Calculate: 𝑑 = sqrt((𝑋1 −𝑌1)^2+ (𝑋2 − 𝑌2)^2+ ...  + (𝑋𝑛 − 𝑌𝑛)^2)
    return np.sqrt(np.sum((np_array1 - np_array2) ** 2))

## Function to assign an entry to a cluster

In [5]:
def assign_to_cluster(entry, centroids):
    """
    Args:
        entry (np.ndarray): the entry to be assigned. It is a 1-dimensional array with n elements.
        centroids (List[np.ndarray]): a list containing the centroids of each cluster. Each centroid is a 1-dimensional NumPy array with n elements. 
    """
    # this variable is used to retorn the cluster 
    cluster = -1
    # assign to min_distance the larger float
    min_distance = sys.float_info.max
    num_clusters = len(centroids)
    for i in range(num_clusters):
        # calculate euclidian distance 
        current_distance = calculate_euclidean_distance(entry, centroids[i])
        if current_distance < min_distance:
            min_distance = current_distance
            cluster = i
    # return the cluster to which the entry belongs and the euclidean distance.
    return cluster, min_distance

## Function to assign all entries to clusters (get clusters)

In [6]:
def get_clusters(matrix, centroids):
    """
    Args: 
        matrix (np.ndarray): an mxn matrix obtained from the pandas dataframe.
        centroids (List[np.ndarray]): a list containing the centroids of each cluster. Each centroid is a 1-dimensional NumPy array with n elements. 
    """
    # declare a list of lists, each inner list represents a different cluster
    clusters = [[] for _ in range(len(centroids))]
    # iterate over the rows
    for i in range(matrix.shape[0]):
        # get the cluster to which the entry belongs.
        cluster_assigned, _ = assign_to_cluster(matrix[i, :], centroids)
        clusters[cluster_assigned].append(i)
    return clusters

## Function to initialize centroids with random numbers 

In [7]:
def init_centroids_random(matrix, centroids): 
    """
    Args: 
        matrix (np.ndarray): an mxn matrix obtained from the pandas dataframe.
        centroids (List[np.ndarray]): a list containing the centroids of each cluster. Each centroid is a 1-dimensional NumPy array with n elements. 
    """
    dimensions = matrix.shape[1] # number of columns
    centroids_count = len(centroids)
    # Iterate through column
    for i in range(dimensions):
        # get the range of column values 
        min_value = np.min(matrix[:, i])
        max_value = np.max(matrix[:, i])
        # Iterate through centroids
        for j in range(centroids_count):
            # generate a random number within the range. 
            centroids[j][i] = np.random.uniform(min_value, max_value)
    return centroids

# K-means algorithm

In [8]:
def recalculate_centroids(matrix, clusters):
    """
    Args: 
        matrix (np.ndarray): an mxn matrix obtained from the pandas dataframe.
        clusters (List[List[]]): a list containing lists representing clusters. 
            Each inner list contains the index of each entry that belongs to the cluster.
    """
    #dimentions = matrix.shape[1]
    n_clusters = len(clusters)
    # declare a list that represent centroids
    centroids = [None] * n_clusters
    for i in range(n_clusters):
        if len(clusters[i]) > 0:
            # get the new centroids calculating the mean across dimensions
            centroids[i] = np.mean(matrix[clusters[i]], axis=0)
        else: 
            # when the cluster do not have entries, we generate random values again
            centroids[i] = []
            for j in range(matrix.shape[1]):
                centroids[i].append(np.random.uniform(np.min(matrix[:, j]), np.max(matrix[:, j])))
    return centroids

In [183]:
def k_means(matrix, k):
    """
    Args: 
        matrix (np.ndarray): an mxn matrix obtained from the pandas dataframe.
        k (int): the number of cluster into which the entries will be clustered
    """
    # get the number of columns in the matrix
    n = matrix.shape[1]
    # declare a list containing np.arrays that represent centroids
    centroids = [np.zeros(n) for _ in range(k)]
    # (1) Initialize centroids with random number
    init_centroids_random(matrix, centroids)
    # repit while the centroids values do not converge
    converged = False
    while not converged:
        # (2) Assign entries to centroids
        clusters = get_clusters(matrix, centroids)
        # (3) Recalculate centroid values
        updated_centroids = recalculate_centroids(matrix, clusters)
        if np.array_equal(centroids, updated_centroids):
            converged = True
        else:
            centroids = updated_centroids
    return clusters, centroids

# Genetic algorithm

In [184]:
def calculate_fitness(matrix, clusters_per_chromosome, chromosome_population):
    """
    Args: 
        matrix (np.ndarray): an mxn matrix obtained from the pandas dataframe.
        clusters_per_chromosome (list[list[list[]]]): contains the entry index per cluster and per chromosome.
        chromosome_population (list[list[np.ndarray]]):  contains the centroids of each cluster per chromosome.
    """
    n_chromosomes = len(chromosome_population)
    n_clusters = len(clusters_per_chromosome[1])
    fitness_per_chromosome = np.zeros(n_chromosomes)
    # Fitness is computed as the sum of the euclidean distance between each entry and the centroid 
    # of the cluster to which it that belongs.
    for i in range(n_chromosomes):
        distance_sum = 0
        for j in range(n_clusters):
            for k in range(len(clusters_per_chromosome[i][j])):
                distance_sum += calculate_euclidean_distance(
                    matrix[clusters_per_chromosome[i][j][k]], chromosome_population[i][j])
        #print(distance_sum)
        if distance_sum != 0:
            fitness_per_chromosome[i] = 1 / distance_sum
    return fitness_per_chromosome

In [185]:
def roulette_wheel_selection(fitness_per_chromosome, chromosome_population):
    """
    Args:
        fitness_per_chromosome (np.ndarray): 1-dimensional array with the fitness values.
        chromosome_population (list[list[np.ndarray]]):  contains the centroids of each cluster per chromosome.
    """
    n_chromosome = len(fitness_per_chromosome)
    selected_chr = [None] * n_chromosome
    fitness_sum = np.sum(fitness_per_chromosome)
    # select the chromosomes:
    for i in range(n_chromosome):
        random_value =  np.random.uniform(0, fitness_sum)
        wheel = 0
        for j in range(n_chromosome):
            wheel += fitness_per_chromosome[j]
            if wheel >= random_value:
                selected_chr[i] = j
                break
    # update the centroids values:
    # declare a list of lists containing np.arrays that represent centroids
    new_chr_population = [None] * n_chromosome
    for i in range(n_chromosome):
        new_chr_population[i] = chromosome_population[selected_chr[i]]
    return new_chr_population

In [186]:
def crossover(chr_population, crossover_prob):
    """
    Args:
        chr_population (list[list[np.ndarray]]):  contains the centroids of each cluster per chromosome.
        crossover_prob (float): a value between 0 and 1 that represents the probability that a pair of adjacent 
            chromosomes exchange information.
    """
    for i in range(0, len(chr_population) - 1, 2):
        random =  np.random.uniform(0, 1)
        if random <= crossover_prob:
            # crossover
            chr_crossover_point = np.random.randint(1, len(chr_population[i]))
            centroid_crossover_point =  np.random.randint(1, len(chr_population[i][0]))
            # first make the exchange at chromosome level, then at centroid level
            chr1 = chr_population[i][:chr_crossover_point] + chr_population[i+1][chr_crossover_point:]
            chr2 = chr_population[i+1][:chr_crossover_point] + chr_population[i][chr_crossover_point:]
            # break the centroid at the crossover point
            parent_centroid1 = chr_population[i][chr_crossover_point]
            parent_centroid2 = chr_population[i+1][chr_crossover_point]
            centroid1 = np.concatenate((parent_centroid1[:centroid_crossover_point], parent_centroid2[centroid_crossover_point:]))
            centroid2 = np.concatenate((parent_centroid2[:centroid_crossover_point], parent_centroid1[centroid_crossover_point:]))
            # assign the new centroids to the chrs
            chr1[chr_crossover_point] = centroid1
            chr2[chr_crossover_point] = centroid2
            # substitute parent chromosomes with new ones
            chr_population[i] = chr1
            chr_population[i+1] = chr2
            #print(chr1)
            #print(chr2)

In [187]:
def mutate(chromosomes_population, mutation_prob):
    """
    Args:
        chr_population (list[list[np.ndarray]]):  contains the centroids of each cluster per chromosome.
        crossover_prob (float): a value between 0 and 1 that represents the mutation probability of a value on a chromosome.
    """
    chr_count = len(chromosomes_population)
    centroids_count = len(chromosomes_population[0])
    dimensions_count = len(chromosomes_population[0][0])
    
    # iterate over chromosomes
    for i in range(chr_count):
        for j in range(centroids_count):
            for k in range(dimensions_count):
                delta =  np.random.uniform(-1, 1)
                if delta <= mutation_prob:
                    # mute using: v ± 2 * δ * v 
                    # according to  Maulik & Bandyopadhyay (2000). 
                    chromosomes_population[i][j][k] = chromosomes_population[i][j][k] + 2 * delta * chromosomes_population[i][j][k]

In [188]:
def genetic_clustering(matrix, k, population_size, crossover_prob, mutation_prob, generation_tolerance):
    """
    Args: 
        matrix (np.ndarray): an mxn matrix obtained from the pandas dataframe.
        k (int): the number of cluster into which the entries will be clustered.
        population_size (int): The number of chromosomes in the population.
        crossover_prob (float): a value between 0 and 1 that represents the probability that a pair of adjacent chromosomes exchange information.
        mutation_prob (float): a value between 0 and 1 that represents the mutation probability of a value on a chromosome.

    """
    # get the number of dimensions / columns in the matrix
    dimensions = matrix.shape[1]
    # declare a list of lists containing np.arrays that represent centroids
    chr_population = [[np.zeros(dimensions) for _ in range(k)] for _ in range(population_size)]
    # declare variables to store the best chromosome found so far
    best_chromosome = None
    clusters_best_chromosome = None
    best_fitness = 0
    # (1) initialize each centroids of the population ramdomly
    for i in range(population_size):
        init_centroids_random(matrix, chr_population[i])
    #print(chr_population)
    
    converged = False
    generations_from_best_fitness = 0
    while not converged:
        clusters_per_chromosome = [None] * population_size
        for i in range (population_size):
            # (2) Assign entries to cluster in each population
            clusters_per_chromosome[i] = get_clusters(matrix, chr_population[i])
            # (3) Recalculate centroids values, according to:
            #    Maulik, U., & Bandyopadhyay, S. (2000). Genetic algorithm-based clustering
            #    technique. Pattern recognition, 33(9), 1455-1465.
            chr_population[i] =  recalculate_centroids(matrix, clusters_per_chromosome[i])
        # (4) calculate the fitness of each chromosome
        # get the fitness in a numpy array
        fitness_per_chromosome = calculate_fitness(matrix, clusters_per_chromosome, chr_population)
        max_fitness = np.max(fitness_per_chromosome)
        if  max_fitness > best_fitness:
            # save the new best chromosome and its clusters
            best_fitness = max_fitness
            print(f'Best fitness: {best_fitness}')
            fitness_index = np.argmax(fitness_per_chromosome)
            best_chromosome = chr_population[fitness_index]
            clusters_best_chromosome = clusters_per_chromosome[fitness_index]
            generations_from_best_fitness = -1
        generations_from_best_fitness += 1
        # determine if algorithm has converged
        if (generations_from_best_fitness >= generation_tolerance):
            converged = True
        else:
            # (5) Select chromosomes using `Roulette wheel selection`, according to:
            #    Maulik, U., & Bandyopadhyay, S. (2000). Genetic algorithm-based clustering
            #    technique. Pattern recognition, 33(9), 1455-1465.
            chr_population = roulette_wheel_selection(fitness_per_chromosome, chr_population)
            # (6) "crossover" the chromosomes
            crossover(chr_population, crossover_prob)
            # (7) "mutate" the chromosomes
            mutate(chr_population, mutation_prob)
    
    # return clusters, centroids and fitness
    return clusters_best_chromosome, best_chromosome, best_fitness

In [181]:
print(genetic_clustering(iris_matrix, 3, 10, 0.1, 0.01, 1000))

Best fitness: 0.008388844473838458
Best fitness: 0.008568470572008505
Best fitness: 0.008817072422476018
Best fitness: 0.00899119593569604
Best fitness: 0.009370521276297006
Best fitness: 0.009394321569239758
Best fitness: 0.00969950299374199
([[100, 101, 102, 103, 104, 105, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 120, 121, 122, 124, 125, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 139, 140, 142, 143, 144, 145, 147, 148, 149], [50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 106, 119, 123, 126, 138, 141, 146], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]], [array([ 6.66976744,  0.81734056, -0.60929777,  0.00929279,  2.        ]), array([5.95438596, 0.01749711, 0.

# Brute force algorithm

In [ ]:
# code

# Tests

**K-means:**

In [ ]:
clusters, _ = k_means(iris_matrix, 32)
print(clusters)

In [ ]:
clusters, _ = k_means(iris_matrix, 4)
print(clusters)

In [ ]:
clusters, _ = k_means(iris_matrix, 3)
print(clusters)

**Genetic algorithm**

In [182]:
print(genetic_clustering(iris_matrix, 3, 10, 0.1, 0.05, 1000))

Best fitness: 0.008584497228454292
Best fitness: 0.009048372225550784
Best fitness: 0.009306114204836088
Best fitness: 0.009357072271225553
Best fitness: 0.009576718014350399
Best fitness: 0.009650873270520229
Best fitness: 0.009866489517609413
Best fitness: 0.00988568664893931
Best fitness: 0.009978742639364038
Best fitness: 0.00997874263936404
([[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 106, 121], [100, 101, 102, 103, 104, 105, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]], [ar